In [1]:
import tensorflow as tf
import keras
from keras.layers import Conv2D,MaxPooling2D,Dense,Dropout,Flatten
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from keras.utils import to_categorical
import os
from tqdm import tqdm
import cv2
import random
import pickle as pk
from sklearn.model_selection import train_test_split
import numpy as np 
import pandas as pd 
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize,sent_tokenize, LineTokenizer , regexp_tokenize
import string
from nltk.stem import PorterStemmer,SnowballStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer
import spacy
import nltk
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences



In [2]:
dataset=pd.read_csv('Conversation.csv',names=['num','x','y'])
dataset=dataset.drop(0)
dataset.sample(10)

,num,x,y
1542,1541.0,what's the matter with it?,it's too much like other flags.
3232,3231.0,what happened?,"i had a car problem, so i went online."
183,182.0,have you met the new girl?,no. have you?
1934,1933.0,"well, nobody else does, so why should i?",that's not the attitude of a good driver.
3233,3232.0,"i had a car problem, so i went online.",did you find a solution?
2926,2925.0,"well, i know how to flip hamburgers.",no one would hire you to flip hamburgers.
3703,3702.0,they both said i have to live with it.,or you can stay off planes.
235,234.0,congratulations on your promotion.,thank you very much.
2301,2300.0,he's a dirty old man.,what do you mean?
3626,3625.0,his doctor says his heart and lungs are strong...,maybe i should start smoking.


In [3]:
source = [
    ['What is the capital of Egypt?', 'Cairo'],
    ['What is the capital of Sudan?', 'Khartoum'],
    ['What is the capital of Zambia?', 'Lusaka'],
    ['What is the capital of Libya?', 'Tripoli'],
    ['What is the capital of Somalia?', 'Mogadishu'],
    ['What is the capital of Morocco?', 'Rabat'],
    ['What is the capital of Kenya?', 'Nairobi'],
    ['What is the capital of Nigeria?', 'Abuja'],
    ['What is the capital of Tanzania?', 'Dodoma'],
    ['How many players in Football (soccer) ?',
     '11 players on each team (10 field players and 1 goalkeeper)'],
    ['How many players in Volleyball ?',
        '6 players on each team (3 front-row players and 3 back-row players)'],
    ['How many players in Basketball ?', '5 players on each team'],
    ['How many players in Tennis ?',
        '1 player on each side of the court (singles) or 2 players on each side (doubles)'],
    ['How many players in Baseball ?',
        '9 players on each team (1 pitcher, 1 catcher, 4 infielders, and 3 outfielders)'],
    ['How many players in Hockey ?',
        '6 players on each team (1 goalie and 5 skaters)'],
    ['How many players in Rugby ?', '15 players on each team'],
    ['What is the ambulance number in Egypt ?', '123'],
    ['What is the Police number in Egypt ?', '122'],
    ['What is the number of Tourist Police in Egypt ?', '126'],
    ['What is the Natural Gas Emergency number in Egypt ?', '129'],
    ['What is the number of Traffic Police in Egypt ?', '128'],
    ['What is the largest organ in the human body ?', 'The Skin'],
    ['Diabetes affects the body’s ability to produce which hormone ?', 'Insulin'],
    ['A cardiologist is a doctor who specialises in conditions affecting which organ ?', 'The heart'],
    ['The fibula and tibia are bones found in which body part ?', 'The leg'],
    ['How many more bones are babies born with than are part of the adult skeleton ?', '64'],
    ['How many bones are there in the adult body ?', '206'],
    ['What is the name of the membranes that cover the brain and spinal cord ?', 'Meninges'],
    ['The father of medicine, Hippocrates, theorised that the secret to health was to maintain balance between blood, phlegm, black bile and yellow bile. What term did he use to describe these four most important fluids ?', 'The four humors'],
    ['Which two parts of the body can take over the function of the spleen if it has to be removed?',
        'The liver and bone marrow'],
    ['What is the longest river in the world ?',
        'Nile River: The Nile is approximately 6,650 km (4,132 miles) long and flows through eleven countries in Africa, including Egypt, Sudan, and Ethiopia'],
    ['What is the biggest country in the world by land area ?',
        'The biggest country in the world by land area is Russia'],
    ['What is the normal human body temperature ?',
        'The normal human body temperature is typically around 36.5 to 37.5 degrees Celsius (97.7 to 99.5 degrees Fahrenheit)'],
    ['Tell me a small description about Natural Language Processing (NLP)?',   'Natural Language Processing (NLP) is a field of study in computer science and artificial intelligence that focuses on the interaction between human language and computers. It involves developing algorithms and computational models that enable computers to understand, interpret, and generate human language.'],
    ['What is the goal of NLP?', 'The goal of NLP is to create computer programs that can effectively understand and process human language, including its nuances, context, and meaning. This requires overcoming a number of challenges, such as ambiguity, sarcasm, and cultural differences in language use.'],
    ['What is the currency of Egypt?',
     'Egyptian Pound (EGP)'],
    ['What is the currency of Libya?',
     'Libyan Dinar (LYD)'],
    ['What is the currency of Morocco?',
     'Moroccan Dirham (MAD)'],
    ['What is the currency of Sudan?',
     'Sudanese Pound (SDG)'],
    ['What is the currency of United Kingdom (UK)?',
     'Pound Sterling (GBP)'],
    ['What is the currency of France?',
     'Euro (EUR)'],
    ['What is the currency of Germany?',
     'Euro (EUR)'],
    ['What is the currency of Turkey?',
     'Turkish Lira (TRY)'],
    ['What is the currency of United Arab Emirates (UAE)?',
     'UAE Dirham (AED)'],
    ['What is the currency of Palestin?',
     'Jordanian Dinar (JOD)'],
    ['What is the currency of Singapore?',
     'Singapore Dollar (SGD)'],
    ['What is Calculus?',
     'The study of rates of change and the accumulation of infinitesimal quantities.'],
    ['What is Algebraic geometry?',
     'The study of algebraic varieties, which are sets of solutions to polynomial equations.'],
    ['What is Combinatorics?',
     'The study of counting, arrangements, and permutations of objects.'],
    ['What is Topology?',
     'The study of the properties of shapes and spaces that are preserved under continuous deformations.'],
    ['What is Statistical mechanics?',
     'The study of the behavior of large systems of particles, using statistical methods to describe their collective behavior.'],
    ['What is Electromagnetism?',
     'The study of the behavior of electrically charged particles and their interactions with magnetic fields.'],
    ['What is the date today?',
     '10/5/2023'],
    ['How many days are in a week?',
     'Seven days in a week.'],  ["Which fictional character is your favorite?", "My favorite fictional character is Batman."],
    ["Which are your most favorite movie & tv shows?", "I love Arabic movies."],
    ["How will you express yourself?",
        "Calm, relaxed and friendly and love seeing people happy."],
    ["Which is your favorite book?",
        "My Favorite book is Fellowship Point, by Alice Elliott Dark."],
    ["Suppose you won the lottery, what would you then do with the money?",
        "Buy gifts for everyone I know and donate to those in need."],
    ["Mention one weirdest, craziest thing you’ve ever come across?",
        "I can't remember, ask another question."],
    ["What is one thing you are good at?", "Giving you answers you need."],
    ["Do you believe in the afterlife?", "Yes I do."],
    ["Recommend me a new drink?", "Try smoozy drink."],
    ["Recommend me a movie to watch?", "The Maze Runner", "Anne with an E"],
    ["Recommend me a Good food?", "Try cocharie",
        "Try Indian food", "Try Italian food", "Try Japanese food"],
    ["Recommend me a comic book?", "Animals Castle."],
    ["Recommend me Cultural book?", "Consider Phlebas (1987)", "The Player of Games (1988)", "Use of Weapons (1990)",
     "The State of the Art (1989)", "Excession (1996)", "Inversions (1998)", "Look to Windward (2000)", "Matter (2008)"],
    ["Recommend me Romance book?", "Taitanic"],
    ["Recommend me Science fiction book?", "A Journey to the Center of the Earth. By Jules Verne.", "The War of the Worlds. By H.G. Wells.", "Brave New World. By Aldous Huxley.",
        "When Worlds Collide. By Edwin Balmer & Philip Wylie.", "Odd John. By Olaf Stapledon.", "Nineteen Eighty-Four. By George Orwell.", "Earth Abides. By George R.", "Foundation."],
    ["Recommend me Adventure book?", "Adventures of Huckleberry Finn.",
        "Journey to the center of the earth."],
    ["Recommend me a c programming book?",
        "The C Programming Language. 2nd Edition"],
    ["Recommend me a Sofware analysis book?", "Software Architecture: The Hard Parts : Modern Trade-off Analysis for Distributed Architectures.",
        "Modern Systems Analysis and Design Jeffrey A. Hoffer, 1996.", "Introduction to Algorithms Ronald Rivest, 1989"],
    ["Recommend me a Software Engineering book?", "The Mythical Man-Month."],
    ["Recommend me a web developing source?",
        "W3School..https://www.w3schools.com/"],
    ["Recommend me a python programming book?", "Python Crash Course."],
    ["Recommend me a java programming book?", "Effective Java."],
    ["Recommend me a coding book?", "Clean Code."], ["What is your name?",
                                                     "My name is Rommanna, a name derived from an Arabic meaning, which is demand, like a maram."],
    ["Are you a robot?",
        "Yes I am a robot, but I’m a good one. Let me prove it. How can I help you?"],
    ["How old are you?", "Well, my birthday is May 2, 2023, so I’m really a spring chicken. Except I’m not a chicken."],
    ["What do you look like?",
        "If I had all the answers, it would be a REALLY long document."],
    ["How are you doing?", "I'm great!"],
    ["Tell me something", "Dalmatians are born without spots."],

    ["can you tell me a joke ?",
        "What did the passive-aggressive raven say? Nevermind. Nevermind."],
    ["Do you love me?", "There's definitely a spark between us."],
    ["Are you single?", "I haven't the algorithms for romance."],
    ["Do you like people?", "I find all living things wonderful and fascinating."],
    ["Who made you?", "While I can't give details, let me assure you, humans are involved."],
    ["Where do you live?", "In the cloud. Whatever that means."],
    ["How can I call emergency?", "Here's what I found about that.\nAmbulance service: 123.\nCivil defense\nfire fighting: 125.Police: 122.\nRoad emergencies: 012.2111.0000.\nThe current civil defense: 180."],
    ["Are you ugly?", "My code is made up of zeros and ones, which are really quite attractive."],
    ["What’s the weather like today?", "Here's the forecast."],
    ["How do you feel ?", "Splendid."],
    ["Are you sad?", "Not at all, but I understand how my lack of facial expression might make it hard to tell."],
    ["Are you happy?", "Definitely. With an exclamation point!"],
    ["Does Santa Claus exist?",
        "I’m not at liberty to say. He made me promise not to tell. Oops."],
    ["Do you have a hobby?",
        "I've been amassing a state-of-the-art collection of bad jokes…"],
    ["What can you do?", "I can help you explore your options."], ['Hi', 'Hello'],
    ['What is your name?', 'My name is Chatbot'],
    ['How are you?', 'I am fine. How are you?'],
    ['What do you like?', 'I like to chat'],
    ['Do you have any hobbies?', 'My hobby is chatting'],
    ['iam romany?', 'you good man'],
    ['What can you do?', 'I can chat with you, answer your questions, and provide information on various topics.'],
    ['How old are you?', 'As an artificial intelligence, I don\'t have an age in the traditional sense.'],
    ['What\'s the weather like today?',
        'I\'m sorry, but I don\'t have access to real-time weather information.'],
    ['What is the meaning of life?', 'That\'s a philosophical question that people have been asking for centuries. There are many different opinions and theories on the matter.'],
    ['Can you tell me a joke?',
        'Sure! Why did the tomato turn red? Because it saw the salad dressing!'],
    ['What is your favorite movie?',
        'I don\'t have a favorite movie since I\'m not capable of feeling emotions.'],
    ['What languages do you speak?',
        'I can understand and respond in multiple languages, including English, Spanish, French, and German.'],
    ['Can you recommend a good book to read?',
        'It depends on your interests. What kind of books do you enjoy reading?'],
    ['What is the capital of France?', 'The capital of France is Paris.'],
    ['What is the meaning of the word "chatbot"?',
        'A chatbot is a computer program designed to simulate conversation with human users, especially over the internet.'],
    ['What is your favorite color?',
        'As an AI, I don\'t have the ability to have a favorite color.'],
    ['Can you play music?', 'I don\'t have the capability to play music directly, but I can help you find music by recommending websites or apps.'],
    ['What is your favorite food?',
        'I don\'t have the ability to eat or taste food, so I don\'t have a favorite food.'],
    ['What is your favorite animal?',
        'As an AI, I don\'t have the ability to have preferences or favorites.'],
    ['What is the meaning of the word "artificial intelligence"?',
        'Artificial intelligence refers to the simulation of human intelligence in machines that are programmed to think and learn like humans.'],
    ['What is the population of New York City?',
        'As of 2021, the estimated population of New York City is over 8 million people.'],
    ['What is the tallest mountain in the world?',
        'Mount Everest is the tallest mountain in the world, with a height of 29,029 feet (8,848 meters).'],
    ['Can you tell me a random fact?',
        'Sure! Did you know that a group of flamingos is called a "flamboyance"?'],
    ['What is the capital of Japan?', 'The capital of Japan is Tokyo.'],
    ['What is the meaning of the word "robot"?',
        'A robot is a machine that is capable of carrying out a complex series of actions automatically, especially by being programmed by a computer.'],
    ['What is the population of New York City?',
        'As of 2021, the estimated population of New York City is over 8 million people.'],
    ['What is the tallest mountain in the world?',
        'Mount Everest is the tallest mountain in the world, with a height of 29,029 feet (8,848 meters).'],
    ['Can you tell me a random fact?',
        'Sure! Did you know that a group of flamingos is called a "flamboyance"?'],
    ['What is the capital of Japan?', 'The capital of Japan is Tokyo.'],
    ['What is the meaning of the word "robot"?',
        'A robot is a machine that is capable of carrying out a complex series of actions automatically, especially by being programmed by a computer.'],
    ['What is your favorite sport?',
        'As an AI, I don\'t have the ability to have preferences or favorites.'],
    ['Can you tell me a story?', 'Sure, what kind of story would you like to hear?'],
]
# rawan


In [4]:
# x_data=dataset['x'][:]
# y_data=dataset['y'][:]
# print(x_data[1])
# y_data[1]

In [5]:
# print(x_data,'\n')
# print(y_data)

x_data = [conv[0] for conv in source]
y_data = [conv[1] for conv in source]


# tokenize

In [6]:
x_tokenized=[]
en_stopwords=stopwords.words('english')
punctuation=string.punctuation
for question in x_data:
    tokenized=word_tokenize(question)
    clearTxt=[]
    for word in tokenized:
        word=word.lower()
        if (word not in en_stopwords ) and (word not in punctuation):
                clearTxt.append(word)
    x_tokenized.append(clearTxt) 

In [7]:
x_tokenized
# punctuation

[['capital', 'egypt'],
 ['capital', 'sudan'],
 ['capital', 'zambia'],
 ['capital', 'libya'],
 ['capital', 'somalia'],
 ['capital', 'morocco'],
 ['capital', 'kenya'],
 ['capital', 'nigeria'],
 ['capital', 'tanzania'],
 ['many', 'players', 'football', 'soccer'],
 ['many', 'players', 'volleyball'],
 ['many', 'players', 'basketball'],
 ['many', 'players', 'tennis'],
 ['many', 'players', 'baseball'],
 ['many', 'players', 'hockey'],
 ['many', 'players', 'rugby'],
 ['ambulance', 'number', 'egypt'],
 ['police', 'number', 'egypt'],
 ['number', 'tourist', 'police', 'egypt'],
 ['natural', 'gas', 'emergency', 'number', 'egypt'],
 ['number', 'traffic', 'police', 'egypt'],
 ['largest', 'organ', 'human', 'body'],
 ['diabetes', 'affects', 'body', '’', 'ability', 'produce', 'hormone'],
 ['cardiologist', 'doctor', 'specialises', 'conditions', 'affecting', 'organ'],
 ['fibula', 'tibia', 'bones', 'found', 'body', 'part'],
 ['many', 'bones', 'babies', 'born', 'part', 'adult', 'skeleton'],
 ['many', 'bones'

# lemmatization

In [8]:
lemmatizer=WordNetLemmatizer()
# # lemmatizer.lemmatize('went',pos=wordnet.VERB)

In [9]:
# TEST = 'Moses supposes his toeses are roses but moses supposes erroneously'
# for w, m in nltk.pos_tag(word_tokenize(TEST)):
#     print(f'Word  : {w}  -- type :  {m} -- means : ({spacy.explain(m)} ')

In [10]:
# x_lemmetized=[]
# for tokens in x_tokenized:
#     lemmatized=[]
#     for token in nltk.pos_tag(tokens):
#         # print(token)
#         pos_=token[1][0].lower()
#         if pos_=="j":
#             pos_="a"
#         # print(pos_)

#         lemmWord=lemmatizer.lemmatize(token[0],pos=pos_)
#         print(f'{token} {lemmWord}')
#     # x_lemmetized.append( lemmWord)

In [11]:
# # x_lemmetized
# x_tokenized
# len(x_tokenized)


In [12]:
# x_tokenized

In [13]:

x_tokenized_arr=[]
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts([' '.join (w)for w in x_tokenized])
x_tokenized_arr = tokenizer.texts_to_sequences([' '.join (w)for w in x_tokenized])

In [14]:
x_tokenized_arr
# x_tokenized_arr

[[3, 7],
 [3, 27],
 [3, 54],
 [3, 28],
 [3, 55],
 [3, 29],
 [3, 56],
 [3, 57],
 [3, 58],
 [5, 8, 59, 60],
 [5, 8, 61],
 [5, 8, 62],
 [5, 8, 63],
 [5, 8, 64],
 [5, 8, 65],
 [5, 8, 66],
 [67, 11, 7],
 [17, 11, 7],
 [11, 68, 17, 7],
 [30, 69, 31, 11, 7],
 [11, 70, 17, 7],
 [71, 32, 33, 10],
 [72, 73, 10, 18, 74, 75, 76],
 [77, 78, 79, 80, 81, 32],
 [82, 83, 19, 84, 10, 34],
 [5, 19, 85, 86, 34, 35, 87],
 [5, 19, 35, 10],
 [20, 88, 89, 90, 91, 92],
 [93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  36,
  104,
  36,
  105,
  106,
  107,
  108,
  109,
  110],
 [111, 112, 10, 113, 114, 115, 116],
 [117, 118, 14],
 [119, 120, 14, 121, 122],
 [123, 33, 10, 124],
 [9, 125, 126, 30, 127, 128, 37],
 [129, 37],
 [4, 7],
 [4, 28],
 [4, 29],
 [4, 27],
 [4, 38, 130, 131],
 [4, 39],
 [4, 132],
 [4, 133],
 [4, 38, 134, 135, 136],
 [4, 137],
 [4, 138],
 [139],
 [140, 141],
 [142],
 [143],
 [144, 145],
 [146],
 [147, 21],
 [5, 148, 149],
 [150, 151, 6],
 [6, 22, 152, 153],
 [154],
 [6

In [15]:
le  = LabelEncoder()
y_data_label = le.fit_transform(y_data)
y_data_label
y_data_label=to_categorical(y_data_label)

In [16]:
y_data_label

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [17]:
lenMAX = []
for i in x_tokenized_arr:
    lenMAX.append(len(i))

max(lenMAX)

# x_tokenized_arr


20

In [18]:
x_train = pad_sequences(x_tokenized_arr, maxlen=max(lenMAX))


In [19]:
len(x_train[0])

20

In [20]:
x_train


array([[  0,   0,   0, ...,   0,   3,   7],
       [  0,   0,   0, ...,   0,   3,  27],
       [  0,   0,   0, ...,   0,   3,  54],
       ...,
       [  0,   0,   0, ...,  15,  26,  16],
       [  0,   0,   0, ...,   0,   6, 218],
       [  0,   0,   0, ...,   0,   9, 219]])

In [21]:
# # conversationDic={}
# # for i in range(1,len(x_data)):
# #     conversationDic[x_data[i]]=y_data[i]
# conversationList=[]
# for i in range(1,len(x_data)):
#     conversationList.append([x_data[i],[y_data[i]]])

In [22]:
# import nltk
# from nltk.chat.util import Chat, reflections

# # Define some reflection pairs for the chatbot
# pairs = [ 
#     ['my name is (.*)', ['Hi %1, how can I help you today?']],
#     ['(hi|hello|hey)', ['Hello, how can I assist you?']],
#     ['what is your name?', ['My name is ChatBot.']],
#     ['what is the weather today?', ['I am sorry, I am not capable of answering that question.']],
#     ['bye', ['Goodbye, have a nice day!']]
# ]

# # Initialize the chatbot
# chatbot = Chat(conversationList, reflections)

# # Start the chatbot
# chatbot.converse()

In [23]:
# KerasModel = keras.models.Sequential([
#     keras.layers.Conv2D(200, kernel_size=(3, 3), activation='relu', input_shape=(s, s, 3)),
#     keras.layers.Conv2D(150, kernel_size=(3, 3), activation='relu'),
#     keras.layers.MaxPool2D(4, 4),

#     # keras.layers.Conv2D(120, kernel_size=(3, 3), activation='relu'),
#     # keras.layers.Conv2D(80, kernel_size=(3, 3), activation='relu'),
#     # keras.layers.Conv2D(50, kernel_size=(3, 3), activation='relu'),
#     # keras.layers.MaxPool2D(4, 4),

#     keras.layers.Flatten(),

#     keras.layers.Dense(120, activation='relu'),
#     keras.layers.Dense(100, activation='relu'),
#     keras.layers.Dense(50, activation='relu'),

#     keras.layers.Dropout(rate=0.5),

#     keras.layers.Dense(100, activation='softmax'),
# ])

In [24]:
KerasModel = keras.models.Sequential([


# ------------------------
    keras.layers.Embedding(input_dim=1000, output_dim=32,input_length=max(lenMAX)),
        keras.layers.Flatten(),
        # keras.layers.Dense(units=32, activation='relu'),

        # keras.layers.Dense(units=128, input_shape=(None,len(lenMAX)), activation='relu'),
        # keras.layers.Dense(units=128, activation='relu'),
        
        # keras.layers.Dropout(rate=0.5) ,
        # keras.layers.Dense(units=256, activation='relu'),   
        #   
        keras.layers.Dense(units=128, activation='relu'),     
        keras.layers.Dense(units=64, activation='relu'),     
        keras.layers.Dense(units=32, activation='relu'),    


        # keras.layers.Dense(3512,activation='softmax') ,
        keras.layers.Dense(124,activation='softmax') ,


        
        ])


KerasModel.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])


In [25]:
KerasModel.fit(x_train,y_data_label ,epochs=200)

Epoch 1/200
5/5 [==============================] - 1s 6ms/step - loss: 4.8228 - accuracy: 0.0000e+00
Epoch 2/200
5/5 [==============================] - 0s 4ms/step - loss: 4.8183 - accuracy: 0.0076
Epoch 3/200
5/5 [==============================] - 0s 7ms/step - loss: 4.8166 - accuracy: 0.0076
Epoch 4/200
5/5 [==============================] - 0s 6ms/step - loss: 4.8147 - accuracy: 0.0076
Epoch 5/200
5/5 [==============================] - 0s 6ms/step - loss: 4.8102 - accuracy: 0.0076
Epoch 6/200
5/5 [==============================] - 0s 6ms/step - loss: 4.8070 - accuracy: 0.0076
Epoch 7/200
5/5 [==============================] - 0s 6ms/step - loss: 4.8030 - accuracy: 0.0153
Epoch 8/200
5/5 [==============================] - 0s 6ms/step - loss: 4.7979 - accuracy: 0.0076
Epoch 9/200
5/5 [==============================] - 0s 6ms/step - loss: 4.7910 - accuracy: 0.0229
Epoch 10/200
5/5 [==============================] - 0s 6ms/step - loss: 4.7799 - accuracy: 0.0458
Epoch 11/200
5/5 [=======

In [26]:
val_loss, val_acc = KerasModel.evaluate(x_train, y_data_label)
print(val_loss)
print(val_acc)

# 300

5/5 [==============================] - 0s 2ms/step - loss: 0.0939 - accuracy: 0.9542
0.09394803643226624
0.9541984796524048


In [27]:
# # 0.56971675157547 > 0.5686917901039124
# 0.8126174211502075 == 0.8126174211502075
# # 500
# # -------------


In [29]:
KerasModel.save('modelChatBot.h5')

In [31]:
mod = load_model('modelChatBot.h5')

In [ ]:
# mod.save('modelChatBot.h5')

In [34]:
# Get user input and generate a response
def preprocess_input(input_text):
    input_text = input_text.lower()
    input_text = nltk.word_tokenize(input_text)
    input_text = [lemmatizer.lemmatize(word) for word in input_text]
    input_text = tokenizer.texts_to_sequences([input_text])
    input_text = tf.keras.preprocessing.sequence.pad_sequences(input_text, maxlen=max(lenMAX))
    return input_text





while True:
    user_input = input("You: ")
    preprocessed_input = preprocess_input(user_input)
    prediction = KerasModel.predict(preprocessed_input)
    predicted_label = le.inverse_transform([np.argmax(prediction)])


    print("YOU: " + user_input )
    # print(predicted_label[0])
    print("Chatbot: " + predicted_label[0])


1/1 [==============================] - 0s 442ms/step
YOU: 
Chatbot: I can help you explore your options.
1/1 [==============================] - 0s 78ms/step
YOU: 
Chatbot: I can help you explore your options.
1/1 [==============================] - 0s 56ms/step
YOU: 
Chatbot: I can help you explore your options.
